In [1]:
! pip install neo4j torch torch-geometric faiss-cpu

In [2]:
import torch
import numpy as np
from torch_geometric.data import Data
from neo4j import GraphDatabase

URI = "neo4j+ssc://2bdf163a.databases.neo4j.io:7687"
AUTH = ("neo4j", "dVRqLgBpDBT3tP37uYphK_zFZRjBHRizDVvRc4LCJRg")

def extract_and_save():
    driver = GraphDatabase.driver(URI, auth=AUTH)
    node_id_map, node_labels, node_meta = {}, [], []
    
    # 1. 온톨로지 정의
    NODE_TYPE_MAP = {'Movie': 0, 'SceneWindow': 1, 'Character': 2, 'NarrativePhase': 3, 'Trope': 4, 'Emotion': 5, 'Setting': 6, 'PersonaArchetype': 7}
    PHASES = ['Exposition', 'Rising', 'Climax', 'Falling', 'Resolution']
    TROPES = ['Revenge', 'Quest', 'Discovery', 'Escape', 'Redemption', 'Sacrifice', 'Betrayal']
    EMOTIONS = ['Joy', 'Trust', 'Fear', 'Surprise', 'Sadness', 'Disgust', 'Anger', 'Anticipation']
    SETTINGS = ['Urban', 'Nature', 'Indoor', 'Tech', 'Historical', 'Surreal', 'Void']
    ARCHETYPES = ['Hero', 'Anti-Hero', 'Mentor', 'Shadow', 'Trickster', 'Rebel', 'Caregiver', 'Lone Wolf']

    with driver.session() as session:
        # 노드 추출
        for label, type_idx in NODE_TYPE_MAP.items():
            result = session.run(f"MATCH (n:{label}) RETURN id(n) AS nid, properties(n) AS props")
            for record in result:
                nid = record["nid"]
                props = record["props"]
                node_id_map[nid] = len(node_id_map)
                node_labels.append(type_idx)
                node_meta.append({'type': label, 'name': props.get('name') or props.get('title') or props.get('script_name'), 
                                  'movie': props.get('movie_title'), 'window': props.get('window_number')})

        # 엣지 추출
        src_list, dst_list, edge_types = [], [], []
        EDGE_TYPE_MAP = {'HAS_WINDOW': 0, 'IN_PHASE': 1, 'HAS_TROPE': 2, 'HAS_EMOTION': 3, 'IN_SETTING': 4, 'APPEARS_IN': 5, 'BELONGS_TO': 6, 'HAS_ARCHETYPE': 7, 'RELATES_TO': 8}
        rels = session.run("MATCH (a)-[r]->(b) RETURN id(a) AS src, id(b) AS dst, type(r) AS rel_type")
        for record in rels:
            if record["src"] in node_id_map and record["dst"] in node_id_map:
                src_list.append(node_id_map[record["src"]])
                dst_list.append(node_id_map[record["dst"]])
                edge_types.append(EDGE_TYPE_MAP.get(record["rel_type"], -1))

    # 2. 43차원 특성 벡터 생성
    x = torch.zeros(len(node_labels), 43)
    for i, (l_idx, meta) in enumerate(zip(node_labels, node_meta)):
        x[i, l_idx] = 1.0 # 기본 타입 원핫
        name = meta['name']
        # 상세 온톨로지 원핫 (가이드 Step 3-3 로직)
        if meta['type'] == 'NarrativePhase' and name in PHASES: x[i, 8 + PHASES.index(name)] = 1.0
        elif meta['type'] == 'Trope' and name in TROPES: x[i, 13 + TROPES.index(name)] = 1.0
        elif meta['type'] == 'Emotion' and name in EMOTIONS: x[i, 20 + EMOTIONS.index(name)] = 1.0
        elif meta['type'] == 'Setting' and name in SETTINGS: x[i, 28 + SETTINGS.index(name)] = 1.0
        elif meta['type'] == 'PersonaArchetype' and name in ARCHETYPES: x[i, 35 + ARCHETYPES.index(name)] = 1.0

    data = Data(x=x, edge_index=torch.tensor([src_list, dst_list], dtype=torch.long),
                edge_type=torch.tensor(edge_types), node_type=torch.tensor(node_labels))
    
    torch.save({'data': data, 'node_meta': node_meta, 'NODE_TYPE_MAP': NODE_TYPE_MAP}, "full_graph.pt")
    print(f"✅ full_graph.pt 생성 완료 (노드: {data.num_nodes})")
    driver.close()

extract_and_save()

/Users/kimdasol/Desktop/recommendation/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=1, column=24, offset=23>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 23, 'line': 1, 'column': 24}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'MATCH (n:Movie) RETURN id(n) AS nid

✅ full_graph.pt 생성 완료 (노드: 5189)


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import faiss
import numpy as np

# 1. 데이터 로드 (보안 옵션 적용)
saved = torch.load("full_graph.pt", weights_only=False)
data = saved['data']
node_meta = saved['node_meta']
NODE_TYPE_MAP = saved['NODE_TYPE_MAP']

# SceneWindow 인덱스 추출
sw_mask = (data.node_type == NODE_TYPE_MAP['SceneWindow'])
sw_indices = sw_mask.nonzero(as_tuple=True)[0]

# 2. GNN 모델 정의 (GAT)
class NarrativeGNN(torch.nn.Module):
    def __init__(self, in_dim=43, hidden_dim=128, out_dim=256):
        super().__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, heads=4, dropout=0.2)
        self.conv2 = GATConv(hidden_dim * 4, out_dim, heads=1, dropout=0.2)
    
    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

model = NarrativeGNN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 3. 학습 루프 (NeighborLoader 없이 전체 그래프 학습)
# 데이터 규모가 크지 않으므로 (~5,400개 노드) 한 번에 학습이 가능합니다.
print("🚀 GNN 학습 시작 (Full-batch 모드)...")
model.train()
for epoch in range(50): 
    optimizer.zero_grad()
    # 전체 그래프 임베딩 생성
    out = model(data.x, data.edge_index)
    
    # SceneWindow 노드들만 추출하여 학습에 사용
    sw_embeddings = out[sw_indices]
    
    # 임시 손실 함수 (실제로는 gnn_guide.md의 contrastive_loss 적용 권장)
    loss = F.mse_loss(sw_embeddings, torch.zeros_like(sw_embeddings))
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/50 | Loss: {loss.item():.4f}")

import os

# 1. 임베딩 추출 및 L2 정규화
model.eval()
with torch.no_grad():
    final_out = model(data.x, data.edge_index)
    sw_embeddings = final_out[sw_indices].cpu().numpy()
faiss.normalize_L2(sw_embeddings)

# 2. 인덱스 생성 및 저장
d = sw_embeddings.shape[1]
new_index = faiss.IndexFlatIP(d)  # Cosine Similarity용
new_index.add(sw_embeddings)

# 파일 저장
faiss.write_index(new_index, "narrative_index.faiss")
np.save("sw_embeddings.npy", sw_embeddings)

print(f"✅ 파일 생성 완료: {os.path.abspath('narrative_index.faiss')}")
print(f"✅ 벡터 개수: {new_index.ntotal}")

🚀 GNN 학습 시작 (Full-batch 모드)...
Epoch 10/50 | Loss: 0.0004
Epoch 20/50 | Loss: 0.0001
Epoch 30/50 | Loss: 0.0001
Epoch 40/50 | Loss: 0.0000
Epoch 50/50 | Loss: 0.0000
✅ 파일 생성 완료: /Users/kimdasol/Desktop/recommendation/narrative_index.faiss
✅ 벡터 개수: 3848
